# Projeto 1 - Ciência dos Dados

Nome: Guilherme Fontana Louro
Nome: Rafael Pascarelli Niccheri

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
/home/guilherme/anaconda3/INSPER/C_dados/Projeto1-Cdados


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
filename = 'ps5.xlsx'

In [4]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Relevância
0,"@jvpmbr we ahem ruim de usar mesmo, não perde ...",0
1,@amandaoufer posso falar como imagino vcs?\nal...,0
2,@ignbrasil @mariananeryf quando sair o trailer...,1
3,@arielsis eu comecei jogar yakuza no meu antig...,1
4,"@psonystas esse ae tá meio doido, tá certo iss...",0


In [5]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Relevância
0,@rigozfn no ps5 foi 5 gb tô dboa,1
1,tamanho das atualizações v16.00:\n\nnintendo s...,1
2,@jhukte eita me dá um ps5 \n\nque eu faço oq v...,0
3,"@pandaescarlate @brksedu @foioguto caralho, va...",0
4,hell let loose será lançado para ps5 em 2021 -...,1


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

O nosso produto é um console de videogame, em sua versão mais atual. Levamos em consideração para definir a relevância dos tweets quesitos como comentários sobre jogos ou funcionalidades do produto, tanto negativas quanto positivas, além de opiniões sobre quesitos que envolvem as vendas desse produto, como locais de venda, preço e principalmente, disponibilidade.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [6]:
import re
import emoji
import functools
import operator


def cleanup(text):
    punctuation = '[!-.:?;@\/|#$%&]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text).lower()
    text_split_emoji = emoji.get_emoji_regexp().split(text_subbed)
    text_split_whitespace = [substr.split() for substr in text_split_emoji]
    text_split = functools.reduce(operator.concat, text_split_whitespace)
    return text_split


In [7]:
def tweet_count(tweets):
    x = 0
    palavras_relevantes = []
    palavras_irrelevantes = []
    while x < len(tweets.Treinamento):
        frase = (cleanup(tweets.Treinamento[x]))
        if tweets.Relevância[x] == 1:
            palavras_relevantes.extend(frase)
        elif tweets.Relevância[x] == 0:
            palavras_irrelevantes.extend(frase)
        x += 1
    serie_relevantes = pd.Series(palavras_relevantes)
    tabela_relevantes = serie_relevantes.value_counts()
    serie_irrelevantes = pd.Series(palavras_irrelevantes)
    tabela_irrelevantes = serie_irrelevantes.value_counts()
    return tabela_relevantes, tabela_irrelevantes
palavras_relevantes = tweet_count(train)[0]
palavras_irrelevantes = tweet_count(train)[1]
palavras = palavras_relevantes + palavras_irrelevantes

In [8]:
print(palavras_relevantes)
print(palavras_relevantes.sum())
print(palavras_irrelevantes)
print(len(palavras))

ps5               144
o                  82
de                 76
e                  66
que                58
                 ... 
🥺                   1
possibilidades      1
trouxe              1
atualizou           1
blaclist            1
Length: 1119, dtype: int64
2905
ps5                   146
o                     108
de                     83
que                    76
e                      76
                     ... 
participarão            1
eh                      1
jogue                   1
interessante            1
httpstco5i2nmkhxrt      1
Length: 1373, dtype: int64
2055


In [9]:
def calculator_porcent(frase):
    x = 0
    frase = (cleanup(frase))
    probFraseRelevante = 1
    probFraseIrrelevante = 1
    while x < len(frase):
        if frase[x] in palavras_relevantes:
            probFraseRelevante *= (palavras_relevantes[frase[x]] + 1) / (palavras_relevantes.sum() + len(palavras))
        elif frase[x] not in palavras_relevantes:
            probFraseRelevante *= 1 / (palavras_relevantes.sum() + len(palavras))
        if frase[x] in palavras_irrelevantes:
            probFraseIrrelevante *= (palavras_irrelevantes[frase[x]] + 1) / (palavras_irrelevantes.sum() + len(palavras))
        elif frase[x] not in palavras_irrelevantes:
            probFraseIrrelevante *= 1 / (palavras_irrelevantes.sum() + len(palavras))
        x += 1
    return probFraseRelevante, probFraseIrrelevante

In [10]:
def relevancia_bot(test):
    y = 0
    probRelevante = []
    probIrrelevante = []
    while y < len(test.Teste):
        probRelevante.append(calculator_porcent(test.Teste[y])[0])
        probIrrelevante.append(calculator_porcent(test.Teste[y])[1])
        y += 1
    z = 0
    Relevânciabot = [0] * len(test)
    while z < len(test.Teste):
        if probRelevante[z] > probIrrelevante[z]:
            Relevânciabot[z] = 1
        z += 1
    test['bot'] = Relevânciabot
    return Relevânciabot
Relevânciabot = relevancia_bot(test)

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [11]:
def calculo_acertos(test):
    acertos = test["Relevância"] == test["bot"]
    test["Acertos"] = acertos
    pd.crosstab(test["Relevância"], test["bot"], normalize='columns')
    estatisticas = pd.crosstab(test.bot,test.Relevância)
    return estatisticas
estatisticas = calculo_acertos(test)

In [12]:
verdadeiro_negativo = estatisticas[0][0] / (estatisticas[0][0] + estatisticas[0][1]) * 100
falso_positivo = estatisticas[0][1] / (estatisticas[0][0] + estatisticas[0][1]) * 100
falso_negativo = estatisticas[1][0] / (estatisticas[1][0] + estatisticas[1][1]) * 100
verdadeiro_positivo = estatisticas[1][1] / (estatisticas[1][0] + estatisticas[1][1]) * 100

In [13]:
print('menssagens irrelevantes classificadas como relevantes: {}%' .format(falso_positivo))
print('menssagens relevantes classificadas como irrelevantes: {}%' .format(falso_negativo))
print('menssagens irrelevantes classificadas como irrelevantes: {}%' .format(verdadeiro_negativo))
print('menssagens relevantes classificadas como relevantes: {}%' .format(verdadeiro_positivo))

menssagens irrelevantes classificadas como relevantes: 56.69291338582677%
menssagens relevantes classificadas como irrelevantes: 39.183673469387756%
menssagens irrelevantes classificadas como irrelevantes: 43.30708661417323%
menssagens relevantes classificadas como relevantes: 60.816326530612244%


___
### Concluindo

A qualidade do classificador fica debilitada pela baixa quantidade de tweets utilizados para fazer a database de relevância. Conseguindo uma taxa de acerto de mais de 60% para verdadeiros positivos e apenas 43% para verdadeiros negativos. Grande parte dos erros são causados por frases de dupla negação ou sarcasmo, pois estas utilizam de palavras normalmente relevantes para criticar irrelevantemente o objeto de estudo, gerando falsos positivos, ou ainda utilizando palavras relacionadas normalmente irrelevantes para comentar sobre a qualidade do produto, gerando falsos negativos. Com um aumento da database, erros como esse acabam diminuindo e o classificador fica mais preciso. Além disso, a qualidade dos tweets não foi a ideal devido a consideração de respostas de tweets anteriores que complicaram a classificação de relevância do projeto.

Para melhorarmos ainda mais o projeto, deveríamos ampliar a quantidade de tweets classificados na base de testes, aprimorando ainda mais a database de palavras relevantes e irrelevantes. Com mais tempo para aperfeiçoar o projeto, podemos ainda definir emojis que retratem sentimentos mais relevantes e aumentarmos assim o valor de frases com esses, além de testar implementações mais complexas do classficador Naive Bayes.

ITEM EXTRA: Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento

RESPOSTA: Não podemos usar o próprio classificador para gerar mais amostras de treinamento porque neste caso as amostras seriam provavelmente frases sem sentido devido a aleatoriedade de suas formações. Além desse fator capital, outro ponto relevante envolve o número limitado de palavras da database, que causariam a falta de vocabulário necessário para criar um número considerável de novas amostras.

ITEM EXTRA: Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer. Indique material de pesquisa sobre o assunto).

RESPOSTA: Assim como sugerido no plano de expansão acima, o projeto de definir emojis mais relevantes seria possível utilizando-se de uma database de emojis bons e ruins, que seriam responsáveis por aumentar ou diminuir o valor de relevância ou irrelevância de determinada frase.
Outra possível ideia envolve a diminuição do número de falsos positivos ou falsos negativos no projeto utilizando da análise de uma curva característica de operação do receptor ou ROC curve, que fornece ferramentas para selecionar modelos mais ideais e desconsiderar modelos mais fracos. Segue links encontrados sobre a ROC curve:

- https://scikit-learn.org/stable/auto_examples/model_selection/plot_roc.html
- https://medium.com/hugo-ferreiras-blog/confusion-matrix-and-other-metrics-in-machine-learning-894688cb1c0a

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)


___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**